In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
engine = create_engine("sqlite:///database.db")
#engine = create_engine("sqlite:///:memory:")

In [3]:
conn = engine.connect()

In [5]:
pd.DataFrame({"first_var": [1, 2],
             "second_var": ["a", "b"]}).to_sql("super_table", conn)

2

In [6]:
pd.read_sql_table("super_table", conn)

,index,first_var,second_var
0,0,1,a
1,1,2,b


In [10]:
pd.read_sql_query(text("SELECT second_var FROM super_table ORDER BY second_var DESC"), conn)

,second_var
0,b
1,a


In [11]:
conn.close()

## Dates and Times

In [12]:
from datetime import date, timedelta
from dateutil.relativedelta import *
from dateutil.parser import *

In [14]:
date.today().weekday()

2

In [17]:
date.today() + relativedelta(months=1)
date.today() + relativedelta(months=-1)
date.today() - relativedelta(months=-1)

datetime.date(2023, 12, 8)

In [19]:
date(2024, 1, 30) + relativedelta(months=1)
date(2024, 2, 29) + relativedelta(months=1)

datetime.date(2024, 3, 29)

In [22]:
date.today() + relativedelta(weeks=1, weekday=TU)

datetime.date(2023, 11, 21)

In [26]:
parse("1 nov 2023")
parse("nov 1 2023")
parse("november 01-2023")
parse("novem 01-2023")

ParserError: Unknown string format: novem 01-2023

In [27]:
markets = pd.read_csv("http://glebvasiliev.com/indexdata.csv", parse_dates=["Date"])
markets2 = pd.read_csv("http://glebvasiliev.com/indexdata.csv")

In [29]:
markets.info()
markets2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104224 entries, 0 to 104223
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Index      104224 non-null  object        
 1   Date       104224 non-null  datetime64[ns]
 2   Open       104224 non-null  float64       
 3   High       104224 non-null  float64       
 4   Low        104224 non-null  float64       
 5   Close      104224 non-null  float64       
 6   Adj Close  104224 non-null  float64       
 7   Volume     104224 non-null  float64       
 8   CloseUSD   104224 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 7.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104224 entries, 0 to 104223
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Index      104224 non-null  object 
 1   Date       104224 non-null  object 
 2   Open       1

In [33]:
markets

(
markets
    .query("Index == 'HSI'")
    .resample("10Y", on="Date")[["Open", "High", "Close", "Low"]]
    .agg(["min", "max"])
)

(
markets
    .groupby("Index")
    .resample("10Y", on="Date")[["Open", "High", "Close", "Low"]]
    .agg(["min", "max", "mean"])
    #.asfreq() 
)

Open                                            \
                               min                                             
                              Open          High         Close           Low   
Index     Date                                                                 
000001.SS 1997-12-31   1064.661011   1100.385986   1041.964966   1025.130005   
          2007-12-31   1007.901001   1019.921997   1011.499023    998.228027   
          2017-12-31   1671.395996   1717.100952   1706.703003   1664.925049   
          2027-12-31   2446.019043   2488.479004   2464.363037   2440.906982   
399001.SZ 1997-12-31   3784.280029   3901.229980   3698.909912   3661.399902   
...                            ...           ...           ...           ...   
SSMI      2030-12-31  10532.309570  10686.019530  10522.219730  10513.429690   
TWII      1997-12-31   7136.240234   7315.819824   7089.560059   7040.540039   
          2007-12-31   3475.870117   3511.379883   3446.260010   3411.679932   
          2017-12-31   3962.689941   4172.490234   4089.929932   3955.429932   
          2027-12-31   8816.860352   9029.519531   8681.339844   8523.629883   

                                                                              \
                               max                                             
                              Open          High         Close           Low   
Index     Date                                                                 
000001.SS 1997-12-31   1260.519043   1264.489014   1256.041016   1245.744019   
          2007-12-31   6057.428223   6124.043945   6092.057129   6040.712891   
          2017-12-31   5507.578125   5522.777832   5497.900879   5456.929199   
          2027-12-31   3721.086914   3731.687012   3696.167969   3663.664063   
399001.SZ 1997-12-31   4706.549805   4755.259766   4679.759766   4646.850098   
...                            ...           ...           ...           ...   
SSMI      2030-12-31  11345.400390  11443.969730  11426.150390  11326.750000   
TWII      1997-12-31  10191.809570  10256.099610  10116.839840  10082.009770   
          2007-12-31  10311.269530  10393.589840  10202.200200  10109.849610   
          2017-12-31  10856.559570  10882.650390  10854.570310  10833.780270   
          2027-12-31  17646.929690  17709.230470  17595.900390  17548.300780   

                                                  ...          Date  \
                              mean                ...           min   
                              Open          High  ...         Close   
Index     Date                                    ...                 
000001.SS 1997-12-31   1169.849337   1179.578937  ...   1041.964966   
          2007-12-31   1800.587073   1816.557357  ...   1011.499023   
          2017-12-31   2790.962193   2817.314759  ...   1706.703003   
          2027-12-31   3053.789902   3074.809044  ...   2464.363037   
399001.SZ 1997-12-31   4313.573607   4356.693426  ...   3698.909912   
...                            ...           ...  ...           ...   
SSMI      2030-12-31  10962.039578  11013.342847  ...  10522.219730   
TWII      1997-12-31   8775.844846   8830.246860  ...   7089.560059   
          2007-12-31   6582.766723   6629.428332  ...   3446.260010   
          2017-12-31   8212.177503   8249.898165  ...   4089.929932   
          2027-12-31  11747.231961  11806.444515  ...   8681.339844   

                                                                              \
                                             max                               
                               Low          Open          High         Close   
Index     Date                                                                 
000001.SS 1997-12-31   1025.130005   1260.519043   1264.489014   1256.041016   
          2007-12-31    998.228027   6057.428223   6124.043945   6092.057129   
          2017-12-31   1664.925049   5507.578125   5522.777832   5497.900879   
  